In [4]:
!pip install lightfm

  Using cached https://files.pythonhosted.org/packages/5e/fe/8864d723daa8e5afc74080ce510c30f7ad52facf6a157d4b42dec83dfab4/lightfm-1.16.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/qq/8_2s6gh54f9g3ytkydqxfsv80000gq/T/pip-install-Np8FoX/lightfm/setup.py", line 3, in <module>
        import pathlib
    ImportError: No module named pathlib
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/qq/8_2s6gh54f9g3ytkydqxfsv80000gq/T/pip-install-Np8FoX/lightfm/
You are using pip version 19.0.3, however version 20.3.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
 
ratings = pd.read_csv('goodbooks-10k/ratings.csv')
books = pd.read_csv('goodbooks-10k/books.csv')
tags = pd.read_csv('goodbooks-10k/tags.csv')
book_tags = pd.read_csv('goodbooks-10k/book_tags.csv')

In [8]:
tags.sample(4)

,tag_id,tag_name
13416,13416,google-play-books
5862,5862,buchanan-renard
33552,33552,الجائزة-العالمية-للرواية-العربية
19301,19301,marketing-selling


In [9]:
book_tags.sample(4)

,goodreads_book_id,tag_id,count
949316,22021611,11491,11
490604,359581,9221,34
847838,15711341,24960,51
84551,10541,30756,7


In [16]:
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [18]:
tags = pd.read_csv('tags_cleaned.csv')
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [19]:
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id)))

In [20]:
#число потоков нашего процессора
NUM_THREADS = 8 

#число параметров вектора 
NUM_COMPONENTS = 30 

#число эпох обучения
NUM_EPOCHS = 10 

In [21]:
#Создаём модель
model = LightFM(learning_rate=0.05, loss='warp', no_components=NUM_COMPONENTS)
 
#Разбиваем наш датасет на обучающую и тестовую выборки
train,test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=None)

#Обучаем модель
model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS,item_features =feature_ratings)

In [22]:
#Тестируем нашу модель
prec_score = precision_at_k(
                     model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()
 
recall_at_k = recall_at_k(model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()

print(recall_at_k,prec_score)

0.037602295696071236 0.08156635


In [23]:
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

In [24]:
!pip install nmslib

    100% |████████████████████████████████| 931kB 2.7MB/s 
  Using cached https://files.pythonhosted.org/packages/f5/7f/a2559a514bdeb2a33e4bf3dc3d2bb17d5acded718893869a82536130cfb3/psutil-5.8.0-cp27-cp27m-macosx_10_9_x86_64.whl
    100% |████████████████████████████████| 194kB 2.2MB/s 
  Using cached https://files.pythonhosted.org/packages/09/96/84cf406fe7d589f3dba9fc0f737e65985a3526c6d8c783f02d4b5a10825d/numpy-1.16.6-cp27-cp27m-macosx_10_9_x86_64.whl
matplotlib 1.3.1 requires nose, which is not installed.
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/Library/Python/2.7/site-packages/psutil'
Consider using the `--user` option or check the permissions.

You are using pip version 19.0.3, however version 20.3.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
import nmslib
 
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [26]:
#Вспомогательная функция для поиска по графу
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn

In [27]:
books[books.original_title.str.find('1984')>=0].head(2)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,...,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...


In [28]:
nbm = nearest_books_nms(846,nms_idx)[0]

In [29]:
books[books.book_id.isin(nbm)]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
12,13,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,...,1956832,2053394,45518,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...
13,14,7613,7613,2207778,896,452284244,9.780452e+12,George Orwell,1945.0,Animal Farm: A Fairy Story,...,1881700,1982987,35472,66854,135147,433432,698642,648912,https://images.gr-assets.com/books/1424037542m...,https://images.gr-assets.com/books/1424037542s...
54,55,5129,5129,3204877,515,60929871,9.780061e+12,Aldous Huxley,1932.0,Brave New World,...,1022601,1079135,20095,26367,60328,219895,389379,383166,https://images.gr-assets.com/books/1487389574m...,https://images.gr-assets.com/books/1487389574s...
288,289,76620,76620,1357456,193,038039586X,9.780380e+12,Richard Adams,1972.0,Watership Down,...,292426,308373,10399,9158,15767,52906,102093,128449,https://images.gr-assets.com/books/1405136931m...,https://images.gr-assets.com/books/1405136931s...
374,375,1852,1852,3252320,1384,439227143,9.780439e+12,Jack London,1903.0,The Call of the Wild,...,223932,248795,6770,6366,16636,62853,90382,72558,https://images.gr-assets.com/books/1452291694m...,https://images.gr-assets.com/books/1452291694s...
528,529,7733,7733,2394716,1762,141439491,9.780141e+12,"Jonathan Swift, Robert DeMaria Jr.",1726.0,Gulliver’s Travels,...,168727,188299,3927,5995,19839,63680,61509,37276,https://images.gr-assets.com/books/1427829692m...,https://images.gr-assets.com/books/1427829692s...
808,809,5479,5479,39947767,38,60776099,9.780061e+12,"Aldous Huxley, Christopher Hitchens",1932.0,Brave New World/Brave New World Revisited,...,108124,110115,1012,1215,3784,18335,39753,47028,https://images.gr-assets.com/books/1331315450m...,https://images.gr-assets.com/books/1331315450s...
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,...,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...
934,935,43035,43035,2949952,1085,439236193,9.780439e+12,Jack London,1906.0,White Fang,...,100223,114519,3017,1823,5850,27264,42175,37407,https://images.gr-assets.com/books/1475878443m...,https://images.gr-assets.com/books/1475878443s...
8139,8140,5481,5481,16335101,80,60898526,9.780061e+12,Aldous Huxley,1958.0,Brave New World Revisited,...,11073,12286,714,231,691,2765,4567,4032,https://images.gr-assets.com/books/1410136964m...,https://images.gr-assets.com/books/1410136964s...


In [34]:
books[books.original_title.str.find('The Silence of the Lambs')>=0].head(2)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
208,209,23807,23807,22533,187,99446782,9.780099e+12,Thomas Harris,1988.0,The Silence of the Lambs,...,351107,366112,3866,10268,12845,55427,123652,163920,https://images.gr-assets.com/books/1390426249m...,https://images.gr-assets.com/books/1390426249s...


In [36]:
nbm = nearest_books_nms(209,nms_idx)[0]
books[books.book_id.isin(nbm)].original_title

208     The Silence of the Lambs
273                The Godfather
430                   Red Dragon
525       Darkly Dreaming Dexter
767               Shutter Island
1484            The Black Dahlia
1801                    Hannibal
3405              The Dante Club
4421             Hannibal Rising
5312               A Simple Plan
Name: original_title, dtype: object

In [37]:
import pickle
with open('item_embeddings.pickle', 'wb') as file:
   pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)